In [1]:
import pandas as pd

In [2]:
df_long = pd.DataFrame({
        "student":
            ["Andy", "Bernie", "Cindy", "Deb",
             "Andy", "Bernie", "Cindy", "Deb",
             "Andy", "Bernie", "Cindy", "Deb"],
        "school":
            ["Z", "Y", "Z", "Y",
             "Z", "Y", "Z", "Y",
             "Z", "Y", "Z", "Y"],
        "class":
            ["english", "english", "english", "english",
             "math", "math", "math", "math",
             "physics", "physics", "physics", "physics"],
        "grade":
            [10, 100, 1000, 10000,
             20, 200, 2000, 20000,
             30, 300, 3000, 30000]

})
df_long

,student,school,class,grade
0,Andy,Z,english,10
1,Bernie,Y,english,100
2,Cindy,Z,english,1000
3,Deb,Y,english,10000
4,Andy,Z,math,20
5,Bernie,Y,math,200
6,Cindy,Z,math,2000
7,Deb,Y,math,20000
8,Andy,Z,physics,30
9,Bernie,Y,physics,300


In [3]:
df_long.pivot_table(index=["student", "school"],
                    columns='class',
                    values='grade').reset_index()

class,student,school,english,math,physics
0,Andy,Z,10,20,30
1,Bernie,Y,100,200,300
2,Cindy,Z,1000,2000,3000
3,Deb,Y,10000,20000,30000


In [4]:
# with aggregation
df_long.pivot_table(index=["student", "school"],
                    columns='class',
                    values='grade',
                    margins=True,
                    aggfunc='sum').reset_index()

class,student,school,english,math,physics,All
0,Andy,Z,10,20,30,60
1,Bernie,Y,100,200,300,600
2,Cindy,Z,1000,2000,3000,6000
3,Deb,Y,10000,20000,30000,60000
4,All,,11110,22220,33330,66660


In [5]:
df_long.pivot_table(index=["student", "school"],
                    aggfunc=['max', 'first'])

max           first       
                  class  grade    class  grade
student school                                
Andy    Z       physics     30  english     10
Bernie  Y       physics    300  english    100
Cindy   Z       physics   3000  english   1000
Deb     Y       physics  30000  english  10000

In [6]:
# fill in blanks
df_long.pivot_table(index="student",
                    columns=['school', 'class'],
                    values='grade',
                    fill_value='.')

school         Y                         Z                
class    english     math  physics english    math physics
student                                                   
Andy           .        .        .    10.0    20.0    30.0
Bernie     100.0    200.0    300.0       .       .       .
Cindy          .        .        .  1000.0  2000.0  3000.0
Deb      10000.0  20000.0  30000.0       .       .       .

Stack/Unstack

In [7]:
# Load necessary libraries
import pandas as pd
import seaborn as sns
import numpy as np

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Enable multiple cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [8]:
tips = sns.load_dataset('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [9]:
# Using groupby
result = tips.groupby('sex')['total_bill'].sum()
result

sex
Male      3256.82
Female    1570.95
Name: total_bill, dtype: float64

In [10]:
result_pivot = tips.pivot_table(values='total_bill', index='sex', aggfunc=np.sum)
result_pivot

,total_bill
sex,
Male,3256.82
Female,1570.95


In [11]:
cars_small = sns.load_dataset('mpg')\
              .set_index('name')[['weight', 'horsepower']].iloc[:10]
cars_small

,weight,horsepower
name,,
chevrolet chevelle malibu,3504,130.0
buick skylark 320,3693,165.0
plymouth satellite,3436,150.0
amc rebel sst,3433,150.0
ford torino,3449,140.0
ford galaxie 500,4341,198.0
chevrolet impala,4354,220.0
plymouth fury iii,4312,215.0
pontiac catalina,4425,225.0


In [12]:
stacked_cars = cars_small.stack()
stacked_cars

name                                 
chevrolet chevelle malibu  weight        3504.0
                           horsepower     130.0
buick skylark 320          weight        3693.0
                           horsepower     165.0
plymouth satellite         weight        3436.0
                           horsepower     150.0
amc rebel sst              weight        3433.0
                           horsepower     150.0
ford torino                weight        3449.0
                           horsepower     140.0
ford galaxie 500           weight        4341.0
                           horsepower     198.0
chevrolet impala           weight        4354.0
                           horsepower     220.0
plymouth fury iii          weight        4312.0
                           horsepower     215.0
pontiac catalina           weight        4425.0
                           horsepower     225.0
amc ambassador dpl         weight        3850.0
                           horsepower     190.0
dt

In [13]:
multi_name = tips.pivot_table(values='total_bill', index=['sex', 'day'],
                              aggfunc=[np.mean, np.median, np.sum])
multi_name

mean     median        sum
            total_bill total_bill total_bill
sex    day                                  
Male   Thur  18.714667     16.975     561.44
       Fri   19.857000     17.215     198.57
       Sat   20.802542     18.240    1227.35
       Sun   21.887241     20.725    1269.46
Female Thur  16.715312     13.785     534.89
       Fri   14.145556     15.380     127.31
       Sat   19.680357     18.360     551.05
       Sun   19.872222     17.410     357.70

In [14]:
multi_name.stack()

mean  median      sum
sex    day                                        
Male   Thur total_bill  18.714667  16.975   561.44
       Fri  total_bill  19.857000  17.215   198.57
       Sat  total_bill  20.802542  18.240  1227.35
       Sun  total_bill  21.887241  20.725  1269.46
Female Thur total_bill  16.715312  13.785   534.89
       Fri  total_bill  14.145556  15.380   127.31
       Sat  total_bill  19.680357  18.360   551.05
       Sun  total_bill  19.872222  17.410   357.70

In [15]:
multi_name.stack(level=0)

total_bill
sex    day                     
Male   Thur mean      18.714667
            median    16.975000
            sum      561.440000
       Fri  mean      19.857000
            median    17.215000
            sum      198.570000
       Sat  mean      20.802542
            median    18.240000
            sum     1227.350000
       Sun  mean      21.887241
            median    20.725000
            sum     1269.460000
Female Thur mean      16.715312
            median    13.785000
            sum      534.890000
       Fri  mean      14.145556
            median    15.380000
            sum      127.310000
       Sat  mean      19.680357
            median    18.360000
            sum      551.050000
       Sun  mean      19.872222
            median    17.410000
            sum      357.700000

In [16]:
stacked_cars.unstack()

,weight,horsepower
name,,
chevrolet chevelle malibu,3504.0,130.0
buick skylark 320,3693.0,165.0
plymouth satellite,3436.0,150.0
amc rebel sst,3433.0,150.0
ford torino,3449.0,140.0
ford galaxie 500,4341.0,198.0
chevrolet impala,4354.0,220.0
plymouth fury iii,4312.0,215.0
pontiac catalina,4425.0,225.0


In [17]:
multiple_groups = tips.groupby(['sex', 'smoker', 'day', 'time'])['total_bill'].sum()
multiple_groups

sex     smoker  day   time  
Male    Yes     Thur  Lunch     191.71
                      Dinner      0.00
                Fri   Lunch      34.16
                      Dinner    129.46
                Sat   Lunch       0.00
                      Dinner    589.62
                Sun   Lunch       0.00
                      Dinner    392.12
        No      Thur  Lunch     369.73
                      Dinner      0.00
                Fri   Lunch       0.00
                      Dinner     34.95
                Sat   Lunch       0.00
                      Dinner    637.73
                Sun   Lunch       0.00
                      Dinner    877.34
Female  Yes     Thur  Lunch     134.53
                      Dinner      0.00
                Fri   Lunch      39.78
                      Dinner     48.80
                Sat   Lunch       0.00
                      Dinner    304.00
                Sun   Lunch       0.00
                      Dinner     66.16
        No      Thur  Lunch     381

In [18]:
multiple_groups.unstack()

time                 Lunch  Dinner
sex    smoker day                 
Male   Yes    Thur  191.71    0.00
              Fri    34.16  129.46
              Sat     0.00  589.62
              Sun     0.00  392.12
       No     Thur  369.73    0.00
              Fri     0.00   34.95
              Sat     0.00  637.73
              Sun     0.00  877.34
Female Yes    Thur  134.53    0.00
              Fri    39.78   48.80
              Sat     0.00  304.00
              Sun     0.00   66.16
       No     Thur  381.58   18.78
              Fri    15.98   22.75
              Sat     0.00  247.05
              Sun     0.00  291.54

In [19]:
multiple_groups.unstack().unstack()

time            Lunch                  Dinner                        
day              Thur    Fri  Sat  Sun   Thur     Fri     Sat     Sun
sex    smoker                                                        
Male   Yes     191.71  34.16  0.0  0.0   0.00  129.46  589.62  392.12
       No      369.73   0.00  0.0  0.0   0.00   34.95  637.73  877.34
Female Yes     134.53  39.78  0.0  0.0   0.00   48.80  304.00   66.16
       No      381.58  15.98  0.0  0.0  18.78   22.75  247.05  291.54